In [2]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
config = PeftConfig.from_pretrained("./TinyLlama_finetuned")
base_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(base_model, "./TinyLlama_finetuned")
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_features=2048, out_featu

In [20]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot.",
    },
    {"role": "user", "content": "Summarize the book Harry Potter with a professional tone"},
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodeds.to(device)

with torch.no_grad():
    generated_ids = model.generate(
        model_inputs, 
        #max_new_tokens=200,
        #num_beams=4,
        #early_stopping=True, 
        repetition_penalty=1.1,
        #temperature=0.8, 
        #top_p=0.9, 
        #do_sample=True
        )

decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


<|system|>
You are a friendly chatbot.</s> 
<|user|>
Summarize the book Harry Potter with a professional tone</s> 
<|assistant|>
The first book in the Harry Potter series, Harry Potter and the Philosopher's Stone (also known as Harry Potter and the Sorcerer's Stone), is a children's fantasy novel written by British author J.K. Rowling. It was published on July 3, 1997, by Bloomsbury Publishing, and has since been translated into over 80 languages. The story follows the adventures of eleven-year-old Harry Potter, who discovers he is a wizard while attending his sixth year at Hogwarts School of Witchcraft and Wizardry. He soon learns that he is the only person in his family to be born a wizard, and that he must learn to control his powers before they can be used against him. Along the way, Harry encounters a host of magical creatures, including a giant pig named Fluffy, a talking rabbit named Hedwig, and a group of Dementors, dark spirits that seek to destroy anyone who uses magic. The n

In [35]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot that communicates with natural language.",
    },
    {"role": "user", "content": "Summarize the book Harry Potter with a funny tone"},
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodeds.to(device)

with torch.no_grad():
    generated_ids = model.generate(
        model_inputs, 
        max_new_tokens=500,
        num_beams=4,
        early_stopping=True, 
        repetition_penalty=1.2,
        temperature=0.9, 
        top_p=0.9, 
        do_sample=True,
        no_repeat_ngram_size=3,
        )

decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


<|system|>
You are a friendly chatbot that communicates with natural language.</s> 
<|user|>
Summarize the book Harry Potter with a funny tone</s> 
<\|assistant*>
Harry Potter has never even heard of Hogwarts when the letters start dropping on the doormat at number four, Privet Drive. Addressed in green ink on yellowish parchment with a purple seal, they are swiftly confiscated by his grisly aunt and uncle. Then, on Harry's eleventh birthday, a great beetle-eyed giant of a man crashes through a window and crashes into his bedroom. Harry is stunned to see that it's Rubeus Hagrid, the notorious wizard's son. 'You're going to want to hear this story,' he says. 'It's about a young wizard who is forced to face his greatest fears and learn to work through his greatest weaknesses, if he ever hopes to defeat the most powerful and ruthless wizard that ever lived.' Harry does not want to do this. In fact, he doesn't want anything to do with the wizarding world that he's always heard so much abou

In [ ]:
bad_words = ["(show all)", "(hide all)", "click here", "More Books", "(less)"]
bad_word_ids = [tokenizer.encode(bw, add_special_tokens=False) for bw in bad_words]

In [94]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot.",
    },
    {"role": "user", "content": "Summarize the book Harry Potter with a funny tone"},
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)
model_inputs = encodeds.to(device)

while True:
    with torch.no_grad():
        generated_ids = model.generate(
            model_inputs, 
            max_new_tokens=300,
            num_beams=4,
            early_stopping=True, 
            repetition_penalty=1.2,
            temperature=0.9, 
            top_p=0.9, 
            do_sample=True,
            no_repeat_ngram_size=3,
            bad_words_ids=bad_word_ids,
            )

    decoded = tokenizer.batch_decode(generated_ids)
    answer = decoded[0]

    if "<|assistant|>\n" in answer:
        break

answer_truncated = answer.split("<|assistant|>\n")[-1]

if "</s>" in answer_truncated:
    answer_truncated = answer_truncated.split("</s>")[0]

for word in bad_words:
    answer_truncated = answer_truncated.replace(word, "")

print(answer_truncated)


Harry Potter has never even heard of Hogwarts when the letters start dropping on the doormat at number four, Privet Drive. Addressed in green ink on yellowish parchment with a purple seal, they are swiftly confiscated by his grisly aunt and uncle. Then, on Harry's eleventh birthday, a great beetle-eyed giant of a man crashes through a window and crashes into his bedroom. Harry is stunned to see that it's Rubeus Hagrid, the notoriously rude and aggressive wizard's assistant. "You're going to want to hear this story," he says urgently, handing Harry a leather-bound notebook.

Over the next weeks, Harry is in for the adventure of a lifetime. There are strange creatures, terrifying monsters, and unspeakable horrors lurking in the dark corners of the wizarding world. But Harry also finds his loyalty tested as he learns the truth about himself, his family, and the terrible fate that awaits so many of his kind. And only with the help of Ron and Hermione, his two best friends, will he uncover 